# Load Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

ImportError: cannot import name 'joblib' from 'sklearn.externals' (c:\users\rishabh\appdata\local\programs\python\python39\lib\site-packages\sklearn\externals\__init__.py)

# Load Custom Classes

In [ ]:
from LoadData.loadData import LoadData

from visual.visual import Graph

from labelling.labels import Labels
from labelling.verify_labels import VerifyLabels


from mlmodel.mlmodel import Models



In [ ]:
# custom class objects
ld_ob = LoadData()

# visualisation objects
vis_ob = Graph()

# label object
lb_ob = Labels()
vlb_ob = VerifyLabels()
# feature objects

# mlmodel object
ml_ob = Models()




# Load Data

In [ ]:
# df = pd.read_csv('./data/plain_road/1.csv')
df = ld_ob.loadData(folder_name='plain_road_unmarked_sb')
# df['Time'] = pd.to_datetime(df['Time'],  format="%Y-%m-%d %H:%M:%S:%f")
print(df.shape)
df.head(10)

# Visualise data

In [ ]:
from plotly import tools
from plotly.offline import download_plotlyjs,init_notebook_mode,iplot,plot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [ ]:
f1,f2 = vis_ob.plot_g_a(df)

# Features

In [ ]:
# code to extract features

# Labels

# Threshold for sampling
## Cusum filter for downsampling

In [ ]:
col_name = 'Az'
ev_vals_df = lb_ob.cusum_filter(df,df[col_name].mean(),col_name)
print(ev_vals_df.shape)

In [ ]:
ev_vals_df.iloc[10:15,:]

# Verify downsampled points

In [ ]:
vlb_ob.verify_samples(df,col_name, ev_vals_df['event'])

In [ ]:
# Set Vertical Limits

vlimits = lb_ob.set_vertical_limits(ev_vals_df)
# print(vlimits.head())

# Set horizontal limits

hlimits = lb_ob.set_horizontal_limits(ev_vals_df)

# Get threshold points

# points above and below h_up and h_dn respectively
thold_pts = lb_ob.get_thold_pts(ev_vals_df,hlimits)
thold_pts.head()

In [ ]:
vlb_ob.verify_vertical_limits(df,col_name, ev_vals_df,ev_vals_df['event'],vlimits)

In [ ]:
vlb_ob.verify_horizontal_limits(df,col_name, ev_vals_df,ev_vals_df['event'],vlimits,hlimits)

# Label Downsampled data

In [ ]:
# set label according to dataset
# 1 : speed breaker
# -1: pothole
labelled_df = lb_ob.get_labels(ev_vals_df,thold_pts,-1)
labelled_df.head()

In [ ]:
labelled_df['label'].value_counts()

In [ ]:
# Verify labels
vlb_ob.verify_labels(df,col_name,labelled_df,labelled_df['event'],vlimits,hlimits)

# Map labels with current dataset

In [ ]:
ix = df['Time'].searchsorted(labelled_df['event'])
ix[0:10]

In [ ]:
df['label'] = np.NaN
label_values = labelled_df['label'].values
df.iloc[ix,-1] = label_values
final_df = df.dropna()
final_df.head(10)

In [ ]:
X = final_df.iloc[:,1:-1]
y = final_df.iloc[:,-1]

# Split

In [ ]:
trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.2, shuffle=False)

trainX.shape, trainY.shape

# Classifier

In [ ]:
clf = ml_ob.randomForest(trainX,trainY,n_estimators=100)

# Predict

In [ ]:
pred_y = clf.predict(testX)

# Cross Validation Metrics

In [ ]:
accuracy_score(testY,pred_y)